#PyroSim

This is a first draft for a simple fire and rescue library

The below class acts and the initiator for the fire resouces. sub-classes can be made which can relate specifcally to certain types of fire resouce or even other emergency services like police or ambulance

In [1]:
import numpy as np
import pandas as pd
from scipy import ndimage
import scipy.signal as signal
from scipy.stats import norm

In [2]:
class Resource:
    def __init__(self, start_hour, target_distance, speed, cost_per_unit, firefighting_capacity, delay_probability_table, resource_id):
        self.cost_per_unit = cost_per_unit
        self.distance_remaining = target_distance
        self.speed = speed
        self.delay_probability_table = delay_probability_table
        self.firefighting_capacity = firefighting_capacity
        self.arrived = False
        self.start_time = start_hour # Start time is the hour the fire started
        self.last_update_time = 0
        self.resource_id = resource_id
        # Initialize arrival_logged to False
        self.arrival_logged = False

    def update(self, current_time):
        # Calculate the elapsed time since the last update
        time_elapsed = current_time - self.last_update_time
        self.last_update_time = current_time
        # If the resource has just arrived, set the start time to the current time
        if self.distance_remaining <= 0 and not self.arrived:
            self.start_time = current_time
            self.arrived = True

        # Check if the resource is delayed
        current_hour = (self.start_time + current_time) % 24
        delay_probability, delay_reduction_factor = self.delay_probability_table[current_hour]

        if np.random.rand() < delay_probability:
            speed = self.speed * delay_reduction_factor
        else:
            speed = self.speed

        # Update the distance remaining
        self.distance_remaining -= speed * time_elapsed

        # Check if the resource has arrived
        if self.distance_remaining <= 0:
            self.arrived = True

    def has_arrived(self):
        return self.arrived

    def get_firefighting_capacity(self):
        return self.firefighting_capacity
    
    def get_cost(self, current_time):
        # The cost is the time the resource has been at the building times the cost per unit
        return (current_time - self.start_time) * self.cost_per_unit

# Building class

The building is initiated as class. this allows it to burn and be modified throughout the simulation

In [3]:
class Building:
    def __init__(self, length, breadth, height, ignition_chance):
        self.dimensions = (length, breadth, height)
        self.ignition_chance = ignition_chance
        # Initialize the fire grid to nan and the extinguished grid to zeros
        self.fire_grid = np.full(self.dimensions, np.nan)
        self.extinguished_grid = np.zeros(self.dimensions)

    def ignite(self, location):
        self.fire_grid[location] = 0

    def spread_fire(self, spread_probability, current_time):
        # Generate random numbers for each cell
        random_grid = np.random.rand(*self.dimensions)

        kernel = np.array([[[0, 0, 0], [1, 1, 1], [0, 0, 0]], 
                        [[0, 1, 0], [1, 0, 1], [0, 1, 0]], 
                        [[0, 0, 0], [1, 1, 1], [0, 0, 0]]])
        
        # Replace NaN values with 0 for convolution
        #fire_grid_nonan = np.where(np.isnan(self.fire_grid), 0, self.fire_grid)
        fire_grid_nonan = np.where(np.isnan(self.fire_grid), 1, self.fire_grid)
        adjacency_mask = signal.convolve(fire_grid_nonan, kernel, mode='same')

        # Set cells on fire if they're adjacent to a burning cell, not already on fire or extinguished, 
        # and the random number is less than the spread probability
        self.fire_grid[(adjacency_mask > 0) & np.isnan(self.fire_grid) & (self.extinguished_grid == 0) & (random_grid < spread_probability)] = current_time



    def extinguish_fire(self, location, current_time):
        self.extinguished_grid[location] = current_time
        self.fire_grid[location] = np.nan

    def fire_magnitude(self):
        return np.count_nonzero(~np.isnan(self.fire_grid))

    def extinguished_magnitude(self):
        return np.count_nonzero(self.extinguished_grid>0)

    def total_state(self):
        return {
            'Dimensions': self.dimensions,
            'Fire Grid': self.fire_grid,
            'Extinguished Grid': self.extinguished_grid,
            'Fire Magnitude': self.fire_magnitude(),
            'Extinguished Magnitude': self.extinguished_magnitude(),
        }
    
    def most_recently_ignited(self, n):
        # Get the locations of all burning cells
        burning_locations = np.argwhere(~np.isnan(self.fire_grid)).tolist()

        # Sort these locations by ignition time, in descending order
        burning_locations = sorted(burning_locations, key=lambda loc: self.fire_grid[tuple(loc)], reverse=True)

        # Return the first 'n' locations
        return burning_locations[:n]
    

    def extinguish_fire(self, firefighting_capacity, current_time, extinguish_prob):
        # Get the locations of the most recently ignited cells, up to the firefighting capacity
        targets = self.most_recently_ignited(firefighting_capacity)

        # Generate a random number for each target
        rand_numbers = np.random.rand(len(targets))

        # Get the indices of the targets that should be extinguished
        extinguish_indices = np.where(rand_numbers < extinguish_prob)[0]

        # Convert self.extinguished_grid and self.fire_grid to numpy arrays if they're not already
        self.extinguished_grid = np.array(self.extinguished_grid)
        self.fire_grid = np.array(self.fire_grid)

        # Create a tuple of arrays for indexing
        targets_array = tuple(np.array(targets)[extinguish_indices].T)

        # Extinguish the selected cells
        self.extinguished_grid[targets_array] = current_time
        self.fire_grid[targets_array] = np.nan


    def get_total_damage(self):
        return np.count_nonzero(self.extinguished_grid>0)

    def get_fraction_damage(self):
        total_cells = np.prod(self.dimensions)
        return self.get_total_damage() / total_cells

    def get_fraction_on_fire(self):
        total_cells = np.prod(self.dimensions)
        on_fire_cells = np.count_nonzero(~np.isnan(self.fire_grid))
        return on_fire_cells / total_cells



In [4]:
# Assuming the classes Building, Resource, and FireSimulation have been defined as per the previous discussions

# Parameters for the building
length = 50
breadth = 50
height = 3
ignition_chance = 0.5
detection_chance = 0.1
spread_probability = 0.01
extinguish_prob = 0.5
# Randomly select a location in the building
random_location = (np.random.randint(0, length), np.random.randint(0, breadth), np.random.randint(0, height))
start_hour = 9

# Parameters for the resources
num_resources = 3
distance = 10  # assuming some distance units
speed = 40/60  # assuming some speed units
cost_per_unit = 100  # assuming some cost units
firefighting_capacity = 70
resources = []

##fire param
prev_fire_size  = 0

# Delay table for the resources
#row number hour of the day
#column A probability of delay
#column B speed deduction factor
delay_table = np.array([
    [0.1, 0.6], [0.1, 0.6], [0.1, 0.6], [0.1, 0.6], [0.1, 0.6], # 00:00 - 05:00
    [0.8, 0.4], [0.8, 0.4], [0.8, 0.4], [0.8, 0.4], # 06:00 - 09:00
    [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5], # 10:00 - 15:00
    [0.8, 0.4], [0.8, 0.4], [0.8, 0.4], [0.8, 0.4], # 16:00 - 19:00
    [0.1, 0.6], [0.1, 0.6], [0.1, 0.6], [0.1, 0.6], [0.1, 0.6]  # 20:00 - 23:00
])
# Minimum reduction fraction for calling additional resources
min_reduction_fraction = 0.1

# Initialize a Building and a set of Resources
building = Building(length, breadth, height, ignition_chance)
# Ignite the location
building.ignite(random_location)

print(building.get_total_damage())
# Set initial time
current_time = 0
fire_fighting_begun = False

# Create an empty DataFrame to hold the simulation events
events = pd.DataFrame(columns=['Time', 'Event', 'Resource ID'])

resource_id = 0

while np.random.rand(1) > detection_chance:
    building.spread_fire(spread_probability, current_time)
    # Increment the current time
    current_time += 1

detection_time = current_time
print(f'Fire detected! Time {current_time}')
events = pd.concat([events, pd.DataFrame([{'Time': current_time, 'Event': 'Fire Detected', 'Resource ID': None}])], ignore_index=True)


for _ in range(num_resources):
    resources.append(Resource(start_hour, distance, speed, cost_per_unit, firefighting_capacity, delay_table, resource_id))
    # Add the dispatch event to the events DataFrame
    events = pd.concat([events, pd.DataFrame([{'Time': current_time, 'Event': 'Resource Dispatched', 'Resource ID': resource_id}])], ignore_index=True)
    resource_id += 1

# Run the simulation until the fire is completely extinguished
while not building.fire_magnitude()==0:
    # Update the fire spread
    building.spread_fire(spread_probability, current_time)

    #print(f"magnitude: {building.fire_magnitude()}")
    #print(f"extinguished: {building.extinguished_magnitude()}")
    #print(current_time)
    # Update the resources
    for resource in resources:
        resource.update(current_time)
        
        if resource.has_arrived() and not resource.arrival_logged:
            resource.arrival_logged = True
            events = pd.concat([events, pd.DataFrame([{'Time': current_time, 'Event': 'Resource Arrived', 'Resource ID': resource.resource_id}])], ignore_index=True)

        # If a resource has just arrived, add an arrival event to the events DataFrame
        if resource.has_arrived():
            building.extinguish_fire(resource.get_firefighting_capacity(), current_time, extinguish_prob)

    # Check if additional resources are needed
    current_fire_size = building.fire_magnitude()
    # Check if all resources have arrived
    resources_in_transit = any(not resource.has_arrived() for resource in resources)

    # Only order more resources if fire size hasn't reduced and no resources are in transit
    if current_fire_size >= prev_fire_size and not resources_in_transit:
        resources.append(Resource(start_hour, distance, speed, cost_per_unit, firefighting_capacity, delay_table, resource_id))  
        events = pd.concat([events, pd.DataFrame([{'Time': current_time, 'Event': 'Additional Resource Dispatched', 'Resource ID': resource_id}])], ignore_index=True)
        resource_id += 1

    # Update prev_fire_size for the next iteration
    prev_fire_size = current_fire_size
    # Increment the current time
    current_time += 1

# Once the fire is extinguished, add an extinguished event to the events DataFrame
events = pd.concat([events, pd.DataFrame([{'Time': current_time, 'Event': 'Fire Extinguished', 'Resource ID': None}])], ignore_index=True)

# Print the results
print(events)


0
Fire detected! Time 11
   Time                           Event Resource ID
0    11                   Fire Detected        None
1    11             Resource Dispatched           0
2    11             Resource Dispatched           1
3    11             Resource Dispatched           2
4    16                Resource Arrived           1
5    16                Resource Arrived           2
6    22                Resource Arrived           0
7    45  Additional Resource Dispatched           3
8    46                Resource Arrived           3
9    49  Additional Resource Dispatched           4
10   50                Resource Arrived           4
11   50  Additional Resource Dispatched           5
12   51                Resource Arrived           5
13   52  Additional Resource Dispatched           6
14   53                Resource Arrived           6
15   53  Additional Resource Dispatched           7
16   54                Resource Arrived           7
17   56               Fire Extinguished